# Deploying BiqQuery ML Model on Vertex AI Model Registry and Make Predictions


## Learning objectives

1. Train a model with BigQuery ML.
2. Upload the model to Vertex AI Model Registry.
3. Create a Vertex AI Endpoint resource.
4. Deploy the Model resource to the Endpoint resource.
5. Make prediction requests to the model endpoint.
6. Run batch prediction job on the Model resource.


## Introduction

This notebook demonstrates how to train a model with BigQuery ML and upload it on Vertex AI Model Registry, then make batch predictions.


### Dataset

The dataset used for this tutorial is the Penguins dataset from <a href="https://cloud.google.com/bigquery/public-data" target="_blank">BigQuery public datasets</a>. This version of the dataset is used to predict the species of penguins from the available features like culmen-length, flipper-depth etc.

Each learning objective will correspond to a __#TODO__ in the notebook, where you will complete the notebook cell's code before running the cell. Refer to the [solution notebook](../solutions/bqml-vertexai-model-registry.ipynb) for reference.

### Install additional packages

Install the following packages required to execute this notebook. 

In [1]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform google-cloud-bigquery pyarrow {USER_FLAG} -q

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [2]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set your project ID

**If you don't know your project ID**, you can get your project ID using `gcloud`.

In [1]:
PROJECT_ID = "[your-project-id]"  # Replace with your Project ID

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [2]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You might not be able to use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about <a href="https://cloud.google.com/vertex-ai/docs/general/locations" target="_blank">Vertex AI regions</a>.

In [3]:
REGION = "[your-region]"  # Replace with your region

if REGION == "[your-region]":
    REGION = "us-central1"

#### UUID

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.

In [4]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### Import libraries

In [5]:
import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery

### Initialize Vertex AI and BigQuery SDKs for Python

Initialize the Vertex AI and Big Query SDKs for Python for your project and corresponding bucket.

In [6]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bqclient = bigquery.Client(project=PROJECT_ID)

## BigQuery ML introduction

BigQuery ML (BQML) provides the capability to train ML tabular models, such as classification and regression, in BigQuery using SQL syntax.

Learn more about <a href="https://cloud.google.com/bigquery-ml/docs" target="_blank">BigQuery ML documentation</a>.

### BigQuery table used for training

In [7]:
# Define BigQuery table to be used for training

BQ_TABLE = "bigquery-public-data.ml_datasets.penguins"

### Create BigQuery dataset resource
First, you create an empty dataset resource in your project.

In [8]:
BQ_DATASET_NAME = "penguins" + UUID
DATASET_QUERY = f"""CREATE SCHEMA {BQ_DATASET_NAME}"""

job = bqclient.query(DATASET_QUERY)
job.result()
print(job.state)

DONE


## Train BigQuery ML model and upload it to Vertex AI Model Registry
Next, you create and train a `BigQuery  ML` tabular regression model from the public dataset penguins and store the model in your project `Vertex AI Model Registry` using the `CREATE MODEL` statement. The model configuration is specified in the `OPTIONS` statement as follows:

- `model_type`: The type and archictecture of tabular model to train, e.g., LOGISTIC_REG.

- `labels`: The column which are the labels.

- `model_registry`: To register a BigQuery ML model to Vertex AI Model Registry, you must use `model_registry="vertex_ai"`.

Learn more about the <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create" target="_blank">CREATE MODEL statement</a>.

Learn more about <a href="https://cloud.google.com/bigquery-ml/docs/managing-models-vertex" target="_blank">Managing BigQuery ML models in the Vertex AI Model Registry</a>.

In [9]:
# TODO 1
# Write the query to create Big Query ML model

MODEL_NAME = "penguins-lr" + UUID
MODEL_QUERY = f"""
# TODO 1: Your code goes here
"""

### Create BigQuery ML Model
Create the BigQuery ML model using the query above and the BigQuery client that you created previously:

In [10]:
# Run the model creation query using BigQuery client

job = bqclient.query(MODEL_QUERY)
print(f"Job state: {job.state}\nJob Error:{job.errors}")

Job state: RUNNING
Job Error:None


Check the job status:

In [11]:
job.result()
print(job.state)

DONE


### Find the model in the Vertex Model Registry

You can use the `Vertex AI Model()` method with `model_name` parameter to find the automatically registered model.

In [12]:
model = aiplatform.Model(model_name=MODEL_NAME)

print(model.gca_resource)

name: "projects/229691399944/locations/us-central1/models/penguins-lrk1wslkxq"
display_name: "penguins-lrk1wslkxq"
supported_deployment_resources_types: DEDICATED_RESOURCES
create_time {
  seconds: 1665674928
  nanos: 102624000
}
update_time {
  seconds: 1665674928
  nanos: 102624000
}
etag: "AMEw9yOPM5CNmD7t1f83OMcYiuneVUgvHusxrh3roBE2Y0Sfvu2vwmyuy0aFFb6LzO4="
version_id: "1"
version_aliases: "default"
version_create_time {
  seconds: 1665674928
  nanos: 102624000
}
version_update_time {
  seconds: 1665675013
  nanos: 157346000
}
model_source_info {
  source_type: BQML
}



## Deploy Vertex AI Model resource to a Vertex AI Endpoint resource
You must deploy a model to an `endpoint` before that model can be used to serve online predictions; deploying a model associates physical resources with the model so it can serve online predictions with low latency. 

Learn more about <a href="https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api#aiplatform_deploy_model_custom_trained_model_sample-python" target="_blank">Deploy a model using the Vertex AI API</a>


### Create a Vertex AI Endpoint resource

If you are deploying a model to an existing endpoint, you can skip this cell.

- `display_name`: Display name for the endpoint.
- `project`: The project ID on which you are creating an endpoint.
- `location`: The region where you are using Vertex AI.

In [13]:
# TODO 2

# Create a Vertex AI endpoint
ENDPOINT_DISPLAY_NAME = "bqml-lr-model-endpoint" + UUID

endpoint = # TODO 2: Your code goes here

print(endpoint.display_name)
print(endpoint.resource_name)

Creating Endpoint
Create Endpoint backing LRO: projects/229691399944/locations/us-central1/endpoints/2771547756227985408/operations/7341815515234435072
Endpoint created. Resource name: projects/229691399944/locations/us-central1/endpoints/2771547756227985408
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/229691399944/locations/us-central1/endpoints/2771547756227985408')
bqml-lr-model-endpointk1wslkxq
projects/229691399944/locations/us-central1/endpoints/2771547756227985408


### Deploy the Vertex AI Model resource to Vertex AI Endpoint resource

In [14]:
# TODO 3
DEPLOYED_NAME = "bqml-lr-penguins"

# Deploy the model using model.deploy method
# TODO 3: Your code goes here

print(model.display_name)
print(model.resource_name)

Deploying model to Endpoint : projects/229691399944/locations/us-central1/endpoints/2771547756227985408
Using default machine_type: n1-standard-2
Deploy Endpoint model backing LRO: projects/229691399944/locations/us-central1/endpoints/2771547756227985408/operations/4542828346823671808
Endpoint model deployed. Resource name: projects/229691399944/locations/us-central1/endpoints/2771547756227985408
penguins-lrk1wslkxq
projects/229691399944/locations/us-central1/models/penguins-lrk1wslkxq


## Send prediction request to the Vertex AI Endpoint resource

Now that your Vertex AI Model resource is deployed to a Vertex AI `Endpoint` resource, you can do online predictions by sending prediction requests to the `Endpoint` resource.

Learn more about <a href="https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-custom-models" target="_blank">Get online predictions from custom-trained models</a>

In [15]:
# TODO 4
instance = {
    "island": "Dream",
    "culmen_length_mm": 36.6,
    "culmen_depth_mm": 18.4,
    "flipper_length_mm": 184.0,
    "body_mass_g": 3475.0,
    "sex": "FEMALE",
}

prediction = # TODO 4: Your code goes here
print(prediction)

Prediction(predictions=[{'species_probs': [0.004134393613856379, 8.708736467695393e-06, 0.9958568976496759], 'predicted_species': ['Adelie Penguin (Pygoscelis adeliae)'], 'species_values': ['Chinstrap penguin (Pygoscelis antarctica)', 'Gentoo penguin (Pygoscelis papua)', 'Adelie Penguin (Pygoscelis adeliae)']}], deployed_model_id='6705354069805891584', model_version_id='1', model_resource_name='projects/229691399944/locations/us-central1/models/penguins-lrk1wslkxq', explanations=None)


## Batch Prediction on the BigQuery ML model
Here you request batch predictions directly from the BigQuery ML model; you don't need to deploy the model to an endpoint. For data types that support both batch and online predictions, use batch predictions when you don't require an immediate response and want to process accumulated data by using a single request.

Learn more abount <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-predict" target="_blank">The ML.PREDICT function</a>

In [16]:
# TODO 5
# Run batch prediction job on the Model
sql_ml_predict = f"""SELECT * FROM ML.PREDICT(MODEL `{PROJECT_ID}.{BQ_DATASET_NAME}.{MODEL_NAME}`, 
(SELECT
      *
    FROM
      `{BQ_TABLE}` LIMIT 10))"""

job = bqclient.query(sql_ml_predict)
prediction_result = # TODO 5: Your code goes here

In [17]:
display(prediction_result.head())

,predicted_species,predicted_species_probs,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Chinstrap penguin (Pygoscelis antarctica),[{'label': 'Chinstrap penguin (Pygoscelis anta...,Chinstrap penguin (Pygoscelis antarctica),Dream,46.9,16.6,192.0,2700.0,FEMALE
1,Adelie Penguin (Pygoscelis adeliae),[{'label': 'Adelie Penguin (Pygoscelis adeliae...,Adelie Penguin (Pygoscelis adeliae),Dream,36.6,18.4,184.0,3475.0,FEMALE
2,Chinstrap penguin (Pygoscelis antarctica),[{'label': 'Chinstrap penguin (Pygoscelis anta...,Chinstrap penguin (Pygoscelis antarctica),Dream,46.5,17.9,192.0,3500.0,FEMALE
3,Adelie Penguin (Pygoscelis adeliae),[{'label': 'Adelie Penguin (Pygoscelis adeliae...,Adelie Penguin (Pygoscelis adeliae),Dream,40.9,18.9,184.0,3900.0,MALE
4,Chinstrap penguin (Pygoscelis antarctica),[{'label': 'Chinstrap penguin (Pygoscelis anta...,Chinstrap penguin (Pygoscelis antarctica),Dream,49.5,19.0,200.0,3800.0,MALE


## Cleaning up

To clean up all Google Cloud resources used in this project, you can <a href="https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects" target="_blank">delete the Google Cloud
project</a> you used for the tutorial.

Learn more about <a href="https://cloud.google.com/bigquery-ml/docs/managing-models-vertex" target="_blank">Deleting BigQuery ML models from Vertex AI Model Registry</a>

Otherwise, you can delete the individual resources you created in this tutorial.

In [18]:
# Delete the endpoint using the Vertex endpoint object
endpoint.undeploy_all()
endpoint.delete()

Undeploying Endpoint model: projects/229691399944/locations/us-central1/endpoints/2771547756227985408
Undeploy Endpoint model backing LRO: projects/229691399944/locations/us-central1/endpoints/2771547756227985408/operations/3468719835695808512
Endpoint model undeployed. Resource name: projects/229691399944/locations/us-central1/endpoints/2771547756227985408
Deleting Endpoint : projects/229691399944/locations/us-central1/endpoints/2771547756227985408
Delete Endpoint  backing LRO: projects/229691399944/locations/us-central1/operations/4151578129195859968
Endpoint deleted. . Resource name: projects/229691399944/locations/us-central1/endpoints/2771547756227985408


In [19]:
# Delete BigQuery ML model

delete_query = f"""DROP MODEL `{PROJECT_ID}.{BQ_DATASET_NAME}.{MODEL_NAME}`"""
job = bqclient.query(delete_query)
job.result()

In [20]:
# Delete the created BigQuery dataset
! bq rm -r -f $PROJECT_ID:$BQ_DATASET_NAME